In [1]:
import pandas as pd
import numpy as np
import os
import requests
import env
import wrangle as w
import explore as exp

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [3]:
# Example usage:
app_token = env.app_token
year_to_retrieve = '2022'
max_req = 2000  # Specify the maximum number of observations to retrieve


In [4]:
# get_data function retrieves and caches the data in csv format ("nyc_collisions_{year}")
# subsequent runs will load the csv as to be respectful of the resources provided for free.
df = w.get_data(year_to_retrieve, app_token)
df.head()
#for year 2022, retrieval took about 00:1:10 (mins) and 103,875 records

CSV file for 2022 already exists. Loading data from the CSV.


,crash_date,crash_time,latitude,longitude,location,on_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,collision_id,vehicle_type_code1,off_street_name,vehicle_type_code2,borough,zip_code,cross_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_4,vehicle_type_code_5
0,2022-01-01T00:00:00.000,7:33,40.771477,-73.91824,"{'latitude': '40.771477', 'longitude': '-73.91...",HOYT AVENUE NORTH,0,0,0,0,0,0,0,0,Unspecified,Unspecified,4491400,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01T00:00:00.000,4:30,NaN,NaN,NaN,Southern parkway,0,0,0,0,0,0,0,0,Pavement Slippery,Unspecified,4491626,Sedan,Jfk expressway,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01T00:00:00.000,7:57,NaN,NaN,NaN,WESTCHESTER AVENUE,0,0,0,0,0,0,0,0,Unspecified,NaN,4491734,Sedan,SHERIDAN EXPRESSWAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01T00:00:00.000,5:17,40.746930,-73.84866,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,1,0,0,0,0,0,1,0,Passing or Lane Usage Improper,Unsafe Lane Changing,4491857,Sedan,NaN,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01T00:00:00.000,1:30,40.819157,-73.96038,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,0,0,0,0,0,0,0,0,Unspecified,NaN,4491344,Sedan,NaN,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58537 entries, 0 to 58536
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_datetime                 58537 non-null  datetime64[ns]
 1   crash_date                     58537 non-null  datetime64[ns]
 2   crash_time                     58537 non-null  object        
 3   collision_id                   58537 non-null  int64         
 4   latitude                       53287 non-null  float64       
 5   longitude                      53287 non-null  float64       
 6   location                       53287 non-null  object        
 7   on_street_name                 45037 non-null  object        
 8   cross_street_name              13498 non-null  object        
 9   off_street_name                29066 non-null  object        
 10  borough                        37671 non-null  object        
 11  zip_code       

In [6]:
# Convert 'crash_date' and 'crash_time' to datetime and combine them into a single column
df['crash_datetime'] = pd.to_datetime(df['crash_date'].str[:10] + ' ' + df['crash_time'], format='%Y-%m-%d %H:%M')
df['crash_date'] = pd.to_datetime(df['crash_date'])


In [7]:
df = w.initial_reorder_cols(df)

In [22]:
# Example usage:
info_dataframe = exp.create_info_dataframe(df)  # Assuming data_df is your DataFrame
info_dataframe

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,49016,datetime64[ns]
1,crash_date,0,0.0,0,0,365,datetime64[ns]
2,crash_time,0,0.0,0,0,1440,object
3,collision_id,0,0.0,0,0,58525,int64
4,latitude,5250,9.0,1013,0,24098,float64
5,longitude,5250,9.0,1013,0,20180,float64
6,location,5250,9.0,0,0,28964,object
7,on_street_name,13500,23.0,0,0,3598,object
8,cross_street_name,45039,77.0,0,0,12342,object
9,off_street_name,29471,50.0,0,0,4096,object


In [17]:
# Create a mask where we filter out any observation with a value in vehicle 3 +
# This will leave us with only the observations that have 2 vehicles involved
condition = (df['vehicle_type_code_3'].isnull() &
             df['vehicle_type_code_4'].isnull() &
             df['vehicle_type_code_5'].isnull() &
             df['contributing_factor_vehicle_3'].isnull()
             )

# Apply the condition to filter the DataFrame
df = df[condition]

# Reset the index if needed
df.reset_index(drop=True, inplace=True)

df.drop(columns=['vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5', 'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5' ], inplace=True)


In [21]:
# filter only keep observations with a value in vehicle_type_code1 and 2
# this will leave us with collisions of 2 vehicles only. 
condition = (df['vehicle_type_code1'].notnull() &
             df['vehicle_type_code2'].notnull())
df = df[condition]

# Reset the index if needed
df.reset_index(drop=True, inplace=True)

In [24]:
df.head(20)

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-01-01 04:30:00,2022-01-01,4:30,4491626,NaN,NaN,NaN,Southern parkway,NaN,Jfk expressway,NaN,NaN,0,0,0,0,0,0,0,0,Sedan,Pavement Slippery,Sedan,Unspecified
1,2022-01-01 05:17:00,2022-01-01,5:17,4491857,40.746930,-73.848660,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,NaN,NaN,NaN,NaN,1,0,0,0,0,0,1,0,Sedan,Passing or Lane Usage Improper,Sedan,Unsafe Lane Changing
2,2022-01-01 01:30:00,2022-01-01,1:30,4491344,40.819157,-73.960380,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,NaN
3,2022-01-01 16:40:00,2022-01-01,16:40,4491478,40.806107,-73.917990,"{'latitude': '40.806107', 'longitude': '-73.91...",NaN,520 EAST 137 STREET,NaN,BRONX,10454.0,0,0,0,0,0,0,0,0,Sedan,Unspecified,Sedan,Unspecified
4,2022-01-01 10:00:00,2022-01-01,10:00,4491791,NaN,NaN,NaN,VANDAM STREET,NaN,BORDEN AVENUE,NaN,NaN,1,0,0,0,0,0,1,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified
5,2022-01-01 02:09:00,2022-01-01,2:09,4491064,40.714996,-73.810220,"{'latitude': '40.714996', 'longitude': '-73.81...",150 STREET,NaN,COOLIDGE AVENUE,QUEENS,11435.0,0,0,0,0,0,0,0,0,Sedan,Following Too Closely,Sedan,Unspecified
6,2022-01-01 02:53:00,2022-01-01,2:53,4491586,40.646034,-73.996780,"{'latitude': '40.646034', 'longitude': '-73.99...",NaN,850 40 STREET,NaN,BROOKLYN,11232.0,0,0,0,0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Unspecified,Station Wagon/Sport Utility Vehicle,Unspecified
7,2022-01-01 17:02:00,2022-01-01,17:02,4491430,40.712360,-73.993750,"{'latitude': '40.71236', 'longitude': '-73.993...",MADISON STREET,NaN,MECHANICS ALLEY,MANHATTAN,10002.0,0,0,0,0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Traffic Control Disregarded,Sedan,Unspecified
8,2022-01-01 03:46:00,2022-01-01,3:46,4491096,40.865795,-73.920006,"{'latitude': '40.865795', 'longitude': '-73.92...",WEST 207 STREET,NaN,SHERMAN AVENUE,MANHATTAN,10034.0,0,0,0,0,0,0,0,0,Taxi,Passing Too Closely,Sedan,Unspecified
9,2022-01-01 18:21:00,2022-01-01,18:21,4491488,40.890648,-73.848750,"{'latitude': '40.890648', 'longitude': '-73.84...",EAST 233 STREET,NaN,GUNTHER AVENUE,BRONX,10466.0,1,0,0,0,0,0,1,0,Sedan,Pavement Slippery,Station Wagon/Sport Utility Vehicle,Unspecified


In [11]:
df.columns

Index(['crash_datetime', 'crash_date', 'crash_time', 'collision_id',
       'latitude', 'longitude', 'location', 'on_street_name',
       'cross_street_name', 'off_street_name', 'borough', 'zip_code',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'vehicle_type_code1', 'contributing_factor_vehicle_1',
       'vehicle_type_code2', 'contributing_factor_vehicle_2',
       'vehicle_type_code_3', 'contributing_factor_vehicle_3',
       'vehicle_type_code_4', 'contributing_factor_vehicle_4',
       'vehicle_type_code_5', 'contributing_factor_vehicle_5'],
      dtype='object')

In [10]:
# Define the time threshold as a string '00:01' (1 minute past midnight)
threshold_time_str = '0:00'

# Filter the DataFrame to include only rows with 'crash_time' before the threshold
filtered_df = df[df['crash_time'] == threshold_time_str]

In [11]:
len(filtered_df)

1615

In [62]:
filtered_df

,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,borough,...,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2,vehicle_type_code_3,contributing_factor_vehicle_3,vehicle_type_code_4,contributing_factor_vehicle_4,vehicle_type_code_5,contributing_factor_vehicle_5
84,2022-01-01T00:00:00.000,0:00,4491456,40.660200,-73.909290,"{'latitude': '40.6602', 'longitude': '-73.9092...",NaN,461 CHESTER STREET,NaN,BROOKLYN,...,Station Wagon/Sport Utility Vehicle,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,2022-01-01T00:00:00.000,0:00,4491675,40.723854,-73.948840,"{'latitude': '40.723854', 'longitude': '-73.94...",NaN,129 ECKFORD STREET,NaN,BROOKLYN,...,Sedan,Passing Too Closely,NaN,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
182,2022-01-01T00:00:00.000,0:00,4491183,40.680492,-73.915695,"{'latitude': '40.680492', 'longitude': '-73.91...",NaN,237 SUMPTER STREET,NaN,BROOKLYN,...,Sedan,Unsafe Speed,Station Wagon/Sport Utility Vehicle,Unspecified,Station Wagon/Sport Utility Vehicle,Unspecified,NaN,NaN,NaN,NaN
408,2022-01-02T00:00:00.000,0:00,4492267,40.862392,-73.918020,"{'latitude': '40.862392', 'longitude': '-73.91...",WEST 205 STREET,NaN,NaN,NaN,...,Sedan,Driver Inattention/Distraction,Sedan,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
468,2022-01-03T00:00:00.000,0:00,4491870,40.816500,-73.946556,"{'latitude': '40.8165', 'longitude': '-73.9465...",8 AVENUE,NaN,NaN,NaN,...,Sedan,Driver Inattention/Distraction,Sedan,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103689,2022-12-31T00:00:00.000,0:00,4594812,40.666070,-73.761750,"{'latitude': '40.66607', 'longitude': '-73.761...",183 STREET,NaN,SOUTH CONDUIT AVENUE,QUEENS,...,Sedan,Driver Inexperience,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103716,2022-12-31T00:00:00.000,0:00,4594801,40.631010,-74.129210,"{'latitude': '40.63101', 'longitude': '-74.129...",POST AVENUE,NaN,DUBOIS AVENUE,STATEN ISLAND,...,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Station Wagon/Sport Utility Vehicle,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
103771,2022-12-31T00:00:00.000,0:00,4594305,40.879047,-73.839600,"{'latitude': '40.879047', 'longitude': '-73.83...",NaN,3419 GRACE AVENUE,NaN,BRONX,...,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified,NaN,Unspecified,NaN,NaN,NaN,NaN
103857,2022-12-31T00:00:00.000,0:00,4597094,40.665916,-73.925470,"{'latitude': '40.665916', 'longitude': '-73.92...",BUFFALO AVENUE,NaN,EAST NEW YORK AVENUE,NaN,...,Station Wagon/Sport Utility Vehicle,Unspecified,Sedan,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df.head()

,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,borough,...,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2,vehicle_type_code_3,contributing_factor_vehicle_3,vehicle_type_code_4,contributing_factor_vehicle_4,vehicle_type_code_5,contributing_factor_vehicle_5,crash_datetime
0,2022-01-01T00:00:00.000,7:33,4491400,40.771477,-73.91824,"{'latitude': '40.771477', 'longitude': '-73.91...",HOYT AVENUE NORTH,NaN,NaN,NaN,...,Unspecified,NaN,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-01 07:33:00
1,2022-01-01T00:00:00.000,4:30,4491626,NaN,NaN,NaN,Southern parkway,NaN,Jfk expressway,NaN,...,Pavement Slippery,Sedan,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-01 04:30:00
2,2022-01-01T00:00:00.000,7:57,4491734,NaN,NaN,NaN,WESTCHESTER AVENUE,NaN,SHERIDAN EXPRESSWAY,NaN,...,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-01 07:57:00
3,2022-01-01T00:00:00.000,5:17,4491857,40.746930,-73.84866,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,NaN,NaN,NaN,...,Passing or Lane Usage Improper,Sedan,Unsafe Lane Changing,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-01 05:17:00
4,2022-01-01T00:00:00.000,1:30,4491344,40.819157,-73.96038,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,NaN,NaN,NaN,...,Unspecified,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-01 01:30:00


In [31]:
selected_columns = df[['crash_datetime', 'crash_date', 'crash_time']][:20]
selected_columns

,crash_datetime,crash_date,crash_time
0,2022-01-01 07:33:00,2022-01-01T00:00:00.000,7:33
1,2022-01-01 04:30:00,2022-01-01T00:00:00.000,4:30
2,2022-01-01 07:57:00,2022-01-01T00:00:00.000,7:57
3,2022-01-01 05:17:00,2022-01-01T00:00:00.000,5:17
4,2022-01-01 01:30:00,2022-01-01T00:00:00.000,1:30
5,2022-01-01 16:40:00,2022-01-01T00:00:00.000,16:40
6,2022-01-01 13:07:00,2022-01-01T00:00:00.000,13:07
7,2022-01-01 10:00:00,2022-01-01T00:00:00.000,10:00
8,2022-01-01 02:09:00,2022-01-01T00:00:00.000,2:09
9,2022-01-01 21:43:00,2022-01-01T00:00:00.000,21:43
